In [ ]:
'''

presigned post: 用來上傳

https://boto3.amazonaws.com/v1/documentation/api/latest/guide/s3-presigned-urls.html#generating-a-presigned-url-to-upload-a-file


presigned url: 用來下載

https://boto3.amazonaws.com/v1/documentation/api/latest/guide/s3-presigned-urls.html


預先簽章的 POST 如同預先簽章的 URL，讓您可以授予使用者寫入權限，而無需提供他們 AWS 登入資料。

'''

In [ ]:
!pip install boto3 awscli requests

In [ ]:
# 啟用客戶端

import boto3

s3_client = boto3.client('s3', endpoint_url='http://localstack-main:4566')

s3_client

In [ ]:
# list s3 buckets
# if there is no bucket in s3, create one.

response = s3_client.list_buckets()

print(response)

print(response.get('Buckets'))
if response.get('Buckets') == []:

    create_bucket_response = s3_client.create_bucket(
        Bucket='cxcxc-aws-certificate',
    )
    
    print(create_bucket_response)


In [ ]:
# 先下載一個檔案，當作上傳練習素材

import requests
import json


url = 'https://jsonplaceholder.typicode.com/posts'
print(f'Beginning file download with requests from {url}')

r = requests.get(url)

with open('./posts.json', 'w') as f:
    json.dump(r.json(), f)

In [ ]:
# 取得 presigned post
# 回傳 dict 內含 url 與 fields 欄位
# 後續上傳物件時發 post request 至 url，並將 fields 連同欲上傳檔案一並提交

presigned_post_response = s3_client.generate_presigned_post(
    Bucket='cxcxc-aws-certificate',
    Key='cxcxc-pre-sign-upload.json',
)

print(presigned_post_response)

In [ ]:
# 使用 request 上傳至 presigned s3 post
# Demonstrate how another Python program can use the presigned URL to upload a file

with open('posts.json', 'r') as f:
    files = {'file': ('posts.json', f)}
    
    http_response = requests.post(
        presigned_post_response['url'], 
        data=presigned_post_response['fields'], 
        files=files
    )

print(http_response)  # 204 No Content: 伺服器成功處理了請求，沒有返回任何內容。

In [ ]:
# list object from bucket 確認是否上傳成功

response = s3_client.list_objects_v2(
    Bucket='cxcxc-aws-certificate',
    MaxKeys=10,
)

response

In [ ]:
# 生成 presigned url，供下載使用

presigned_url = s3_client.generate_presigned_url('get_object',
    Params={'Bucket': 'cxcxc-aws-certificate',
            'Key': 'cxcxc-pre-sign-upload.json'},
)

print(presigned_url)

In [ ]:
# 透過request模組進行下載

import requests
import json

url = 'http://localstack-main:4566/cxcxc-aws-certificate/cxcxc-pre-sign-upload.json'
print(f'Beginning file download with requests from {presigned_url}')

r = requests.get(presigned_url)
print(r)

with open('./cxcxc-pre-sign-url.json', 'w') as f:
    json.dump(r.json(), f)
    